In [4]:
import wikipedia as wp
import numpy as np
import pandas as pd
from geopy.geocoders import Nominatim
import requests
from sklearn.cluster import KMeans
import folium
import io

pd.set_option('display.max_columns',None)
pd.set_option('display.max_columns',None)

#Create dataframe of postal codes
html = wp.page("List of postal codes of Canada: M").html().encode("UTF-8")
df = pd.read_html(html)[0]
df = df[df.Borough != 'Not assigned']

#Add latitude and longitude coordinates
url="https://cocl.us/Geospatial_data"
s = requests.get(url).content
c = pd.read_csv(io.StringIO(s.decode('UTF-8')))
df = df.merge(c, on='Postal Code')

#K-means algorithm
k=5
kmeans = KMeans(n_clusters=k)
df['Cluster'] = kmeans.fit_predict(df[['Latitude','Longitude']])

#Create map template
location = Nominatim(user_agent="ny_explorer").geocode('Toronto')
latitude = location.latitude
longitude = location.longitude
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

#Assign colors to map
colors=['blue','red','green','yellow','purple']
for lat, lng, borough, neighborhood, cluster in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighbourhood'], df['Cluster']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color=colors[cluster],
        fill=True,
        fill_color=colors[cluster],
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)

map_toronto